# Caching LLM responses

## Colab-specific setup

Make sure you have a Database, a Secure Connect Bundle and a Database Token. See [DB Setup](https://cassio.org/db_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.
Get ready to upload the Bundle and supply the Token string.

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly. See [API Setup](https://cassio.org/api_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.

_Note: this colab is autogenerated from a [regular Jupyter notebook](https://cassio.org/frameworks/langchain/caching-llm-responses/) hosted on cassio.org. Run all cells in this section to complete the setup before moving on to the demo content proper._

_Note: you can work with your own Cassandra cluster instead of Astra DB, provided it is reachable from the cloud: check the commented code below for the case `cqlMode="local"` and consult [cassio.org](https://cassio.org/frameworks/langchain/setup/#database-choice) for more details._

In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio#egg=langchain" \
    "cassandra-driver>=3.28.0" \
    "cassio>=0.0.2" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_APPLICATION_TOKEN = input('Your Astra DB Token: ')

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# Set your secret(s) for LLM access (key names must match `providerValidator` in `llm_choice`)
llmProvider = 'OpenAI'  # 'VertexAI'
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

ASTRA_DB_CLIENT_ID = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_APPLICATION_TOKEN,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    # elif mode == 'local':
    #     cluster = Cluster(
    #         ['192.168.0.1', '192.168.0.2'],
    #         auth_provider=PlainTextAuthProvider(
    #             "username",
    #             "password!",
    #         ),
    #     )
    #     # See https://docs.datastax.com/en/developer/python-driver/latest/getting_started/#connecting-to-cassandra for more options
    #     localSession = cluster.connect()
    #     return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    # elif mode == 'local':
    #     return <NAME_OF_YOUR_LOCAL_KEYSPACE>
    else:
        raise ValueError('Unknown CQL Session mode')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# Caching LLM responses

This notebook demonstrates how to use Cassandra for a basic prompt/response cache.

Such a cache prevents running an LLM invocation more than once for the very same prompt, thus saving on latency and token usage. The cache retrieval logic is based on an exact match, as will be shown.

In [1]:
from langchain.cache import CassandraCache

In [2]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

Create a `CassandraCache` and configure it globally for LangChain:

In [3]:
import langchain
langchain.llm_cache = CassandraCache(
    session=session,
    keyspace=keyspace,
)

In [4]:
langchain.llm_cache.clear()

Below is the logic to instantiate the LLM of choice. We choose to leave it in the notebooks for clarity.

In [5]:
# creation of the LLM resources


if llmProvider == 'VertexAI':
    from langchain.llms import VertexAI
    llm = VertexAI()
    print('LLM from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    llm = OpenAI()
    print('LLM from OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM from OpenAI


In [6]:
%%time
SPIDER_QUESTION_FORM_1 = "How many eyes do spiders have?"
# The first time, it is not yet in cache, so it should take longer
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 18 ms, sys: 1.39 ms, total: 19.4 ms
Wall time: 2.25 s


'\n\nMost spiders have eight eyes, although some have six or fewer.'

In [7]:
%%time
# This time we expect a much shorter answer time
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 2.2 ms, sys: 0 ns, total: 2.2 ms
Wall time: 32.2 ms


'\n\nMost spiders have eight eyes, although some have six or fewer.'

In [8]:
%%time
SPIDER_QUESTION_FORM_2 = "How many eyes do spiders generally have?"
# This will again take 1-2 seconds, being a different string
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 13.4 ms, sys: 992 µs, total: 14.4 ms
Wall time: 1.17 s


'\n\nSpiders typically have eight eyes.'

### Stale entry control

#### Time-To-Live (TTL)

You can configure a time-to-live property of the cache, with the effect of automatic eviction of cached entries after a certain time.

Setting `langchain.llm_cache` to the following will have the effect that entries vanish in an hour:

In [9]:
cacheWithTTL = CassandraCache(
    session=session,
    keyspace=keyspace,
    ttl_seconds=3600,
)

#### Manual cache eviction

Alternatively, you can invalidate cached entries one at a time - for that, you'll need to provide the very LLM this entry is associated to:

In [10]:
%%time
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 3.96 ms, sys: 0 ns, total: 3.96 ms
Wall time: 31.3 ms


'\n\nSpiders typically have eight eyes.'

In [11]:
langchain.llm_cache.delete_through_llm(SPIDER_QUESTION_FORM_2, llm)

In [12]:
%%time
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 9.78 ms, sys: 1.18 ms, total: 11 ms
Wall time: 1.45 s


'\n\nSpiders typically have eight eyes, though some species may have fewer or more.'

#### Whole-cache deletion

As you might have seen at the beginning of this notebook, you can also clear the cache entirely: **all** stored entries, for all models, will be evicted at once:

In [13]:
langchain.llm_cache.clear()